[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/langchain/agents/langchain-weaviate-query-agent.ipynb)

## Weaviate Query Agent with LangChain

This notebook will show you how to define the Weaviate Query Agent as a tool with LangChain.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install LangChain with `pip install langchain`. We're using version `0.3.20` at the time of writing this notebook.
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [2]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.chat_models import init_chat_model

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

### Initialize model

In [4]:
llm = init_chat_model("gpt-4o", model_provider="openai")

### Create Query Agent tool

In [7]:
@tool
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.
    
    Args:
        query (str): Any question or query to search for in the database
        
    Returns:
        str: The response from the database containing relevant information
    """
    # Initialize Weaviate client
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"),
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY") # add the API key to the model provider from your Weaviate collection
            },
    )
    
    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["WeaviateBlogChunks"]
    )
        
    print("Weaviate client and Blogs collection initialized successfully")

    return query_agent.run(query)

In [8]:
llm_with_tools = llm.bind_tools([send_query_agent_request])

### Query time

In [9]:
query = "How do I run Weaviate with Docker?"

llm_with_tools.invoke(query)

AIMessage(content='To run Weaviate with Docker, you can follow these steps:\n\n1. **Install Docker**: Make sure Docker is installed on your machine. You can download it from the [Docker website](https://www.docker.com/products/docker-desktop).\n\n2. **Pull Weaviate Image**: Open your terminal and pull the Weaviate Docker image from Docker Hub using the following command:\n   ```bash\n   docker pull semitechnologies/weaviate\n   ```\n\n3. **Run Weaviate Container**: You can run Weaviate using the following command:\n   ```bash\n   docker run -d -p 8080:8080 semitechnologies/weaviate\n   ```\n   This command will run Weaviate in a detached mode (`-d`), mapping the container\'s port 8080 to your local machine\'s port 8080.\n\n4. **Check if Running**: To verify if the Weaviate container is running, you can use:\n   ```bash\n   docker ps\n   ```\n   This command lists all running containers. You should see the Weaviate container listed there.\n\n5. **Access Weaviate**: Once the container is